# Libraries

In [1]:
import pandas as pd 
import numpy as numpy
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import requests
from seleniumbase import Driver
from selenium.common.exceptions import NoSuchElementException
import pycountry as pc
from geopy.geocoders import Nominatim

#  Scrapping & Cleaning

In [2]:
#Check if Automation is Detected
driver = Driver(uc=True)
try:
    driver.get("https://nowsecure.nl/#relax")
    driver.sleep(4)
finally:
    driver.quit()

**VIU**



In [3]:
driver = Driver(uc=True)
driver.get()
username = ''
password = ''
driver.find_element(By.XPATH,'//*[@id="signin-username"]').send_keys(username)
driver.find_element(By.XPATH,'//*[@id="signin-password"]').send_keys(password)
driver.find_element(By.XPATH,'//*[@id="content"]/form/p[4]/input').send_keys(Keys.ENTER)
#redirect to a page
driver.get('')
#change the option from 3 months to beginning of time
#Select the option
select = Select(driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[2]/select'))
select.select_by_visible_text('Beginning of time')

total_elements = len(driver.find_elements(By.CSS_SELECTOR, '#content_results > div'))
data = []



In [5]:
total_elements = len(driver.find_elements(By.CSS_SELECTOR, '#content_results > div'))
data = []

# Get the total number of elements
total_elements = len(driver.find_elements(By.CSS_SELECTOR, '#content_results > div'))
total_elements

for i in range(1, total_elements + 1):
    xpath = '//*[@id="content_results"]/div[' + str(i) + ']/div[2]'
    elements = driver.find_element(By.XPATH, xpath)
    data.append(elements.text)

df = pd.DataFrame(data, columns=['Information'])


df[['Title','Rating','Year']] = df['Information'].str.split('\n',expand=True)

df.drop(columns=['Rating','Year','Information'],inplace=True)
driver.quit()


In [6]:
df

,Title
0,The Witcher
1,Two and a Half Men
2,Dawn of the Planet of the Apes
3,The Last Stop in Yuma County
4,Dark Matter
...,...
1483,The Commando
1484,Rumble
1485,Red Notice
1486,Clifford the Big Red Dog


**MBP** 

In [7]:
driver2 = Driver(uc=True)
driver2.get('')
time.sleep(20)
driver2.get('')

data2 = []

for i in range(1, 14):
    hist_page = '' + str(i)  # Make sure to define your_base_url correctly
    driver2.get(hist_page)
    
    j = 1
    while j <= 9:
        xpath = '/html/body/div/div/div[6]/div[2]/div[' + str(j) + ']'
        try:
            element = driver2.find_element(By.XPATH, xpath)
            data2.append(element.text)
            j += 1
        except NoSuchElementException:
            break  # Exit the while loop if element is not found

df2 = pd.DataFrame(data2, columns=['Information'])
df2[['Time','Title','Device','DatenTime']] = df2['Information'].str.split('\n',expand=True)
df2.drop(columns=['Information','Time','DatenTime','Device'],inplace=True)
df2

,Title
0,X-Men '97
1,Dark Matter
2,Sugar
3,Abbott Elementary
4,Star Trek: Discovery
...,...
97,Teen Wolf: The Movie
98,Harry Potter and the Deathly Hallows: Part 2
99,We Own This City
100,The Boondocks


**MAL**

In [8]:
driver3 = Driver(uc=True)
driver3.get('https://myanimelist.net/animelist/Tjkill3r?status=7')



data3 = []
driver3 = Driver(uc=True)
driver3.get('https://myanimelist.net/animelist/Tjkill3r?status=7')

for i in range(2,175):
    xpath = '//*[@id="list-container"]/div[4]/div/table/tbody['+str(i)+']/tr[1]/td[4]/a'
    elements = driver3.find_element(By.XPATH,xpath)
    data3.append(elements.text)
driver3.quit()

df3 = pd.DataFrame(data3, columns=['Title']) 

**Netflix**

In [9]:
df4 = pd.read_csv('netflix.csv')

import re

def extract_show_name(title):
    # Check if the title contains both part after ':' and '(episode number)'
    if ":" in title and re.search(r'\(Episode \d+\)', title):
        # Get the first part before ':'
        show_name = title.split(":")[0].strip()
        
        return show_name
    
    # Check if the title contains "Limited Series"
    if "Limited Series" in title:
        return title.split(":")[0].strip()
    
    # Check if the title contains "Season"
    if "Season" in title:
        # Split the title by ":"
        parts = title.split(":")
        
        # Remove the season and episode information
        show_name = ":".join(parts[:-1]).strip()
        
        # Remove the episode number if present
        show_name = re.sub(r'\(Episode \d+\)$', '', show_name).strip()
        
        # Remove the season information if present
        show_name = re.sub(r': Season \d+', '', show_name).strip()
        
        return show_name
    
    # Check if the title contains "Series"
    if "Series" in title:
        # Split the title by ":"
        parts = title.split(":")
        
        # Remove the series information
        show_name = ":".join(parts[:-1]).strip()
        
        return show_name
    
    # Return the original title if it doesn't match any of the conditions
    return title

# Apply the function to the title column
df4['Title'] = df4['Title'].apply(extract_show_name)

df4.drop_duplicates(subset=['Title'],inplace=True)


**OLD MovieBox Data**

In [10]:
df5 = pd.read_csv('moviebox_v1.csv')

In [11]:
df6 = pd.concat([df,df2,df3,df4,df5],ignore_index=True)


In [12]:
df6

,Title
0,The Witcher
1,Two and a Half Men
2,Dawn of the Planet of the Apes
3,The Last Stop in Yuma County
4,Dark Matter
...,...
4112,Crossroads
4113,Nine Perfect Strangers
4114,Charlie's Angels
4115,Locked Up


# Data Retrieval from TMDB

In [13]:
import requests
import pandas as pd

# Define the base URL and API key
base_url = "https://api.themoviedb.org/3"
api_key = ""

# Initialize an empty list to store show data
content_data = []

# List of priority providers
priority_providers = ["Amazon Prime Video", "Netflix", "Disney Plus", "Apple TV Plus"]

# Fetch genre details for both TV shows and movies
tv_genre_url = f"{base_url}/genre/tv/list?api_key={api_key}"
tv_genre_response = requests.get(tv_genre_url)
tv_genre_data = tv_genre_response.json()
tv_genres = {genre['id']: genre['name'] for genre in tv_genre_data['genres']}

movies_genre_url = f"{base_url}/genre/movie/list?api_key={api_key}"
movies_genre_response = requests.get(movies_genre_url)
movies_genre_data = movies_genre_response.json()
movie_genres = {genre['id']: genre['name'] for genre in movies_genre_data['genres']}


for content in df6['Title']:
    # Search for the content by name
    search_url = f"{base_url}/search/multi?api_key={api_key}&query={content}"
    search_response = requests.get(search_url)
    search_data = search_response.json()

    # Check if the search results are not empty
    if search_data['results']:
        # Get the first result, which is the most relevant one
        first_result = search_data['results'][0]

        # Extract required details
        content_id = first_result.get('id')
        title = first_result.get('name') or first_result.get('title')
        poster = first_result.get('poster_path')
        backdrop = first_result.get('backdrop_path')
        overview = first_result.get('overview')
        content_type = first_result.get('media_type', 'N/A')
        rating = first_result.get('vote_average')
        content_genres_ids = first_result.get('genre_ids', [])
        release_year = first_result.get('first_air_date' if content_type == 'tv' else 'release_date', None)

        # Find genre names
        genres = tv_genres if content_type == 'tv' else movie_genres
        content_genres = [genres.get(genre_id, 'Unknown') for genre_id in content_genres_ids]

        #Trailers link
        trailer_url = f"{base_url}/{content_type}/{content_id}/videos?api_key={api_key}"
        trailer_response = requests.get(trailer_url)
        trailer_data = trailer_response.json()
        trailer_key = trailer_data.get('results', [{}])[0].get('key', 'N/A') if trailer_data.get('results') else 'N/A'
        trailer_link = f"https://www.youtube.com/watch?v={trailer_key}" if trailer_key != 'N/A' else 'N/A'


        # Fetch where to watch providers for the content in the UK region
        watch_url = f"{base_url}/{content_type}/{content_id}/watch/providers?api_key={api_key}"
        watch_response = requests.get(watch_url)
        watch_data = watch_response.json()
        watch_providers = watch_data.get('results', {}).get('GB', {})
        providers_info = watch_providers.get('flatrate', [])

        content_details = f"{base_url}/{content_type}/{content_id}?api_key={api_key}"
        content_response = requests.get(content_details)
        content_infos = content_response.json()
        num_episodes = content_infos.get('number_of_episodes', None)
        if content_type == 'tv':
            runtime = content_infos.get('episode_run_time', [])
            runtime = runtime[0] if runtime else None  # Get the first runtime if the list isn't empty
        else:
            runtime = content_infos.get('runtime', None)

        # Get the origin country
        if content_type == 'tv':
            origin = content_infos.get('origin_country','N/A')
        if content_type == 'tv':
            origin = first_result.get('origin_country', 'N/A')
        if content_type == 'movie':
            if content_infos.get('production_countries') and len(content_infos['production_countries']) > 0:
                origin = content_infos['production_countries'][0].get('iso_3166_1', 'N/A')
            else:
                origin = 'N/A'
        if content_type == 'movie':
            if content_infos.get('production_companies') and len(content_infos['production_companies']) > 0:
                origin = content_infos['production_companies'][0].get('origin_country', 'N/A')
        else:
            origin = first_result.get('origin_country', 'N/A')

        # If no valid runtime has been found and the content type is 'tv', check 'last_episode_to_air'
        if runtime is None and content_type == 'tv':
            runtime = content_infos.get('last_episode_to_air', {}).get('runtime', None)

        if runtime is not None:
            if isinstance(runtime,list):
             runtime = runtime[0] if runtime else None
            else:
             runtime = runtime
        else:
            runtime = None
        # Prioritize specific providers
        selected_provider = None
        for provider in providers_info:
            if provider['provider_name'] in priority_providers:
                selected_provider = provider
                break


        # If no priority provider is found, use the first available provider
        if not selected_provider and providers_info:
            selected_provider = providers_info[0]

        # Create a dictionary for the content
        content_info = {
            'Title': title,
            'Content ID': content_id,
            'Poster': poster,
            'Backdrop':backdrop,
            'Overview': overview,
            'Trailer': trailer_link,
            'Type': content_type,
            'Genres': ', '.join(content_genres),
            'Runtime (minutes)': runtime,
            'Number of Episodes': num_episodes if content_type == 'tv' else 1,
            'Rating': rating,
            'Release Year': release_year.split('-')[0] if release_year else None,
            'Provider Name': selected_provider['provider_name'] if selected_provider else 'N/A',
            'Provider Logo': selected_provider['logo_path'] if selected_provider else 'N/A',
            'Origin Country': origin
        }

        print('Processing', title)
        # Append to the list
        content_data.append(content_info)
        total = len(df6['Title'])
        print("Remaining:",total - len(content_data))

# Convert the list to a DataFrame
df_content = pd.DataFrame(content_data)
print(df_content)


Processing The Witcher
Remaining: 4116
Processing Two and a Half Men
Remaining: 4115
Processing Dawn of the Planet of the Apes
Remaining: 4114
Processing The Last Stop in Yuma County
Remaining: 4113
Processing Dark Matter
Remaining: 4112
Processing War for the Planet of the Apes
Remaining: 4111
Processing Dune: Part Two
Remaining: 4110
Processing Living with Leopards
Remaining: 4109
Processing S.W.A.T.
Remaining: 4108
Processing Mothers' Instinct
Remaining: 4107
Processing Thelma the Unicorn
Remaining: 4106
Processing Rise of the Planet of the Apes
Remaining: 4105
Processing Texas Metal's Loud and Lifted
Remaining: 4104
Processing First Dates NL
Remaining: 4103
Processing Bottoms
Remaining: 4102
Processing OMG Fashun
Remaining: 4101
Processing I Kissed a Girl
Remaining: 4100
Processing The Bold and the Beautiful
Remaining: 4099
Processing We're Here
Remaining: 4098
Processing Strictly Confidential
Remaining: 4097
Processing Challengers
Remaining: 4096
Processing The Proof Is Out There:

In [14]:
import requests
import pandas as pd

# Define the base URL and API key
base_url = "https://api.themoviedb.org/3"
api_key = "9121de210a58fbaa7cd5f0654a7ec8d9"

# Initialize an empty list to store show data
content_data_2 = []

# List of priority providers
priority_providers = ["Amazon Prime Video", "Netflix", "Disney Plus", "Apple TV Plus"]

# Fetch genre details for both TV shows and movies
tv_genre_url = f"{base_url}/genre/tv/list?api_key={api_key}"
tv_genre_response = requests.get(tv_genre_url)
tv_genre_data = tv_genre_response.json()
tv_genres = {genre['id']: genre['name'] for genre in tv_genre_data['genres']}

movies_genre_url = f"{base_url}/genre/movie/list?api_key={api_key}"
movies_genre_response = requests.get(movies_genre_url)
movies_genre_data = movies_genre_response.json()
movie_genres = {genre['id']: genre['name'] for genre in movies_genre_data['genres']}


for content in ['One Piece','Avatar: The Last Airbender']:
    # Search for the content by name
    search_url = f"{base_url}/search/multi?api_key={api_key}&query={content}"
    search_response = requests.get(search_url)
    search_data = search_response.json()

    # Check if the search results are not empty
    if search_data['results']:
        # Get the first result, which is the most relevant one
        first_result = search_data['results'][1]

        # Extract required details
        content_id = first_result.get('id')
        title = first_result.get('name') or first_result.get('title')
        poster = first_result.get('poster_path')
        backdrop = first_result.get('backdrop_path')
        overview = first_result.get('overview')
        content_type = first_result.get('media_type', 'N/A')
        rating = first_result.get('vote_average')
        origin = first_result.get('origin_country')
        content_genres_ids = first_result.get('genre_ids', [])
        release_year = first_result.get('first_air_date' if content_type == 'tv' else 'release_date', None)

        # Find genre names
        genres = tv_genres if content_type == 'tv' else movie_genres
        content_genres = [genres.get(genre_id, 'Unknown') for genre_id in content_genres_ids]

        #Trailers link
        trailer_url = f"{base_url}/{content_type}/{content_id}/videos?api_key={api_key}"
        trailer_response = requests.get(trailer_url)
        trailer_data = trailer_response.json()
        trailer_key = trailer_data.get('results', [{}])[0].get('key', 'N/A') if trailer_data.get('results') else 'N/A'
        trailer_link = f"https://www.youtube.com/watch?v={trailer_key}" if trailer_key != 'N/A' else 'N/A'


        # Fetch where to watch providers for the content in the UK region
        watch_url = f"{base_url}/{content_type}/{content_id}/watch/providers?api_key={api_key}"
        watch_response = requests.get(watch_url)
        watch_data = watch_response.json()
        watch_providers = watch_data.get('results', {}).get('GB', {})
        providers_info = watch_providers.get('flatrate', [])

        content_details = f"{base_url}/{content_type}/{content_id}?api_key={api_key}"
        content_response = requests.get(content_details)
        content_infos = content_response.json()
        num_episodes = content_infos.get('number_of_episodes', None)

        # If no valid runtime has been found and the content type is 'tv', check 'last_episode_to_air'

        # Get the origin country
        if content_type == 'tv':
            origin = content_infos.get('origin_country','N/A')[0]
        if content_type == 'movie':
            if content_infos.get('production_countries') and len(content_infos['production_countries']) > 0:
                origin = content_infos['production_countries'][0].get('iso_3166_1', 'N/A')
            else:
                origin = 'N/A'
        if content_type == 'movie':
            if content_infos.get('production_companies') and len(content_infos['production_companies']) > 0:
                origin = content_infos['production_companies'][0].get('origin_country', 'N/A')
        else:
            origin = first_result.get('origin_country', 'N/A')
        if content_type == 'tv':
            runtime = content_infos.get('episode_run_time', [])
            runtime = runtime[0] if runtime else None  # Get the first runtime if the list isn't empty
        else:
            runtime = content_infos.get('runtime', None)

        if runtime is None and content_type == 'tv':
            runtime = content_infos.get('last_episode_to_air', {}).get('runtime', None)


        if runtime is not None:
            if isinstance(runtime,list):
             runtime = runtime[0] if runtime else None
            else:
             runtime = runtime
        else:
            runtime = None
        # Prioritize specific providers
        selected_provider = None
        for provider in providers_info:
            if provider['provider_name'] in priority_providers:
                selected_provider = provider
                break


        # If no priority provider is found, use the first available provider
        if not selected_provider and providers_info:
            selected_provider = providers_info[0]

        # Create a dictionary for the content
        content_info = {
            'Title': title,
            'Content ID': content_id,
            'Poster': poster,
            'Backdrop':backdrop,
            'Overview': overview,
            'Trailer': trailer_link,
            'Type': content_type,
            'Genres': ', '.join(content_genres),
            'Runtime (minutes)': runtime,
            'Number of Episodes': num_episodes if content_type == 'tv' else 1,
            'Rating': rating,
            'Release Year': release_year.split('-')[0] if release_year else None,
            'Provider Name': selected_provider['provider_name'] if selected_provider else 'N/A',
            'Provider Logo': selected_provider['logo_path'] if selected_provider else 'N/A',
            'Origin Country': origin
        }

        print('Processing', title)
        # Append to the list
        content_data_2.append(content_info)

# Convert the list to a DataFrame
df_2 = pd.DataFrame(content_data_2)
print(df_2)


Processing One Piece
Processing Avatar: The Last Airbender
                        Title  Content ID                            Poster  \
0                   One Piece       37854  /cMD9Ygz11zjJzAovURpO75Qg7rT.jpg   
1  Avatar: The Last Airbender       82452  /lzZpWEaqzP0qVA5nkCc5ASbNcSy.jpg   

                           Backdrop  \
0  /2rmK7mnchw9Xr3XdiTFSxTTLXqv.jpg   
1  /imlTCObfzISogbvcwB1dokoXAIc.jpg   

                                            Overview  \
0  Years ago, the fearsome Pirate King, Gol D. Ro...   
1  A young boy known as the Avatar must master th...   

                                       Trailer Type  \
0  https://www.youtube.com/watch?v=LrCmsOAgvxk   tv   
1  https://www.youtube.com/watch?v=xEmlE9Ianuo   tv   

                                        Genres  Runtime (minutes)  \
0        Action & Adventure, Comedy, Animation                 24   
1  Sci-Fi & Fantasy, Action & Adventure, Drama                 58   

   Number of Episodes  Rating Release Year

In [15]:
import requests
import pandas as pd

# Define the base URL and API key
base_url = "https://api.themoviedb.org/3"
api_key = "9121de210a58fbaa7cd5f0654a7ec8d9"

# Initialize an empty list to store show data
content_data_3 = []

# List of priority providers
priority_providers = ["Amazon Prime Video", "Netflix", "Disney Plus", "Apple TV Plus"]

# Fetch genre details for both TV shows and movies
tv_genre_url = f"{base_url}/genre/tv/list?api_key={api_key}"
tv_genre_response = requests.get(tv_genre_url)
tv_genre_data = tv_genre_response.json()
tv_genres = {genre['id']: genre['name'] for genre in tv_genre_data['genres']}

movies_genre_url = f"{base_url}/genre/movie/list?api_key={api_key}"
movies_genre_response = requests.get(movies_genre_url)
movies_genre_data = movies_genre_response.json()
movie_genres = {genre['id']: genre['name'] for genre in movies_genre_data['genres']}


for content in ['Ajin']:
    # Search for the content by name
    search_url = f"{base_url}/search/multi?api_key={api_key}&query={content}"
    search_response = requests.get(search_url)
    search_data = search_response.json()

    # Check if the search results are not empty
    if search_data['results']:
        # Get the first result, which is the most relevant one
        first_result = search_data['results'][1]

        # Extract required details
        content_id = first_result.get('id')
        title = first_result.get('name') or first_result.get('title')
        poster = first_result.get('poster_path')
        backdrop = first_result.get('backdrop_path')
        overview = first_result.get('overview')
        content_type = first_result.get('media_type', 'N/A')
        rating = first_result.get('vote_average')
        origin = first_result.get('origin_country')
        content_genres_ids = first_result.get('genre_ids', [])
        release_year = first_result.get('first_air_date' if content_type == 'tv' else 'release_date', None)

        # Find genre names
        genres = tv_genres if content_type == 'tv' else movie_genres
        content_genres = [genres.get(genre_id, 'Unknown') for genre_id in content_genres_ids]

        #Trailers link
        trailer_url = f"{base_url}/{content_type}/{content_id}/videos?api_key={api_key}"
        trailer_response = requests.get(trailer_url)
        trailer_data = trailer_response.json()
        trailer_key = trailer_data.get('results', [{}])[0].get('key', 'N/A') if trailer_data.get('results') else 'N/A'
        trailer_link = f"https://www.youtube.com/watch?v={trailer_key}" if trailer_key != 'N/A' else 'N/A'


        # Fetch where to watch providers for the content in the UK region
        watch_url = f"{base_url}/{content_type}/{content_id}/watch/providers?api_key={api_key}"
        watch_response = requests.get(watch_url)
        watch_data = watch_response.json()
        watch_providers = watch_data.get('results', {}).get('GB', {})
        providers_info = watch_providers.get('flatrate', [])

        content_details = f"{base_url}/{content_type}/{content_id}?api_key={api_key}"
        content_response = requests.get(content_details)
        content_infos = content_response.json()
        num_episodes = content_infos.get('number_of_episodes', None)
        if content_type == 'tv':
            runtime = content_infos.get('episode_run_time', [])
            runtime = runtime[0] if runtime else None  # Get the first runtime if the list isn't empty
        else:
            runtime = content_infos.get('runtime', None)

        # If no valid runtime has been found and the content type is 'tv', check 'last_episode_to_air'
        if runtime is None and content_type == 'tv':
            runtime = content_infos.get('last_episode_to_air', {}).get('runtime', None)


        if runtime is not None:
            if isinstance(runtime,list):
             runtime = runtime[0] if runtime else None
            else:
             runtime = runtime
        else:
            runtime = None
        # Prioritize specific providers
        selected_provider = None
        for provider in providers_info:
            if provider['provider_name'] in priority_providers:
                selected_provider = provider
                break
        if content_type == 'tv':
            origin = content_infos.get('origin_country','N/A')[0]
        else: 
            origin = content_infos.get('production_companies', [{}])[0].get('origin_country', 'N/A')


        if content_type == 'tv':
            runtime = content_infos.get('episode_run_time', [])
            runtime = runtime[0] if runtime else None  # Get the first runtime if the list isn't empty
        else:
            runtime = content_infos.get('runtime', None)

        # If no priority provider is found, use the first available provider
        if not selected_provider and providers_info:
            selected_provider = providers_info[0]

        # Create a dictionary for the content
        content_info = {
            'Title': title,
            'Content ID': content_id,
            'Poster': poster,
            'Backdrop':backdrop,
            'Overview': overview,
            'Trailer': trailer_link,
            'Type': content_type,
            'Genres': ', '.join(content_genres),
            'Runtime (minutes)': runtime,
            'Number of Episodes': num_episodes if content_type == 'tv' else 1,
            'Rating': rating,
            'Release Year': release_year.split('-')[0] if release_year else None,
            'Provider Name': selected_provider['provider_name'] if selected_provider else 'N/A',
            'Provider Logo': selected_provider['logo_path'] if selected_provider else 'N/A',
            'Origin Country': origin
        }

        print('Processing', title)
        # Append to the list
        content_data_3.append(content_info)

# Convert the list to a DataFrame
df_3 = pd.DataFrame(content_data_3)
print(df_3)


Processing Ajin
  Title  Content ID                           Poster  \
0  Ajin       65292  /fnHBwIKxwrFleSzvHzTqcAzDRM.jpg   

                           Backdrop  \
0  /kOQkt6e18Fe5h9qnkd3PmffX7og.jpg   

                                            Overview  \
0  17 years ago, immortals first appeared on the ...   

                                       Trailer Type  \
0  https://www.youtube.com/watch?v=OO21Sq_CB0A   tv   

                                   Genres  Runtime (minutes)  \
0  Animation, Action & Adventure, Mystery                 24   

   Number of Episodes  Rating Release Year Provider Name  \
0                  26   8.014         2016       Netflix   

                      Provider Logo Origin Country  
0  /pbpMk2JmcoNnQwx5JGpXngfoWtp.jpg             JP  


In [16]:
final_content = pd.concat([df_content,df_3,df_2],ignore_index=True)

**Cleaning**

In [17]:

#IN the Genres column select all the data before the comma and put into a new column Primary Genres
final_content['Primary Genres'] = final_content['Genres'].str.split(',').str[0]
#Secondary Genre will be the second genre in the Genres column after the comma
final_content['Secondary Genres'] = final_content['Genres'].str.split(',').str[1]
round(final_content['Rating'],2)
final_content['Rating'] = round(final_content['Rating'],2)
final_content
#fill blank with N/A
final_content.fillna('N/A',inplace=True)


In [18]:
import pandas as pd
#Where applicable select the first value in the list and  skip anything that is not a list
def extract_first_(entry):
    if isinstance(entry,list) and len(entry) > 0:
        return entry[0]
    return entry
   
final_content['Origin Country'] = final_content['Origin Country'].apply(extract_first_)
# Removing brackets from the 'Origin Country' column
final_content['Origin Country'] = final_content['Origin Country'].astype(str).str.replace('[\[\]]',"" , regex=True)
# Remove single and double quotes
final_content['Origin Country'] = final_content['Origin Country'].str.replace("[\"']", '', regex=True)

# Trim whitespace
final_content['Origin Country'] = final_content['Origin Country'].str.strip()

final_content['Origin Country'].unique()




array(['US', 'GB', 'NL', 'KN', 'AU', 'BR', '', 'JP', 'CA', 'KR', 'FR',
       'N/A', 'CN', 'NZ', 'MY', 'ES', 'IT', 'DE', 'IE', 'CL', 'SE', 'RU',
       'CO', 'ZA', 'DK', 'AR', 'PL', 'AT', 'UA', 'IN', 'FI', 'HK', 'BG',
       'TR', 'EG', 'PT', 'NO', 'CH', 'BE', 'MA', 'NG', 'HU', 'MX', 'LV',
       'KE', 'PH', 'PK', 'TH', 'XC', 'AE', 'ID', 'CZ', 'HR', 'IS', 'LU',
       'RS'], dtype=object)

In [19]:
import pycountry as pyc
def get_country_name(code):
    try:
        country = pc.countries.get(alpha_2=code)
        return country.official_name
    except:
        return 'N/A'

final_content['Origin Country'] = final_content['Origin Country'].apply(get_country_name)
final_content


C:\Users\tedjo\AppData\Roaming\Python\Python311\site-packages\pycountry\db.py:51: UserWarning: Country's official_name not found. Country name provided instead.
  warnings.warn(warning_message, UserWarning)


,Title,Content ID,Poster,Backdrop,Overview,Trailer,Type,Genres,Runtime (minutes),Number of Episodes,Rating,Release Year,Provider Name,Provider Logo,Origin Country,Primary Genres,Secondary Genres
0,The Witcher,71912,/cZ0d3rtvXPVvuiX22sP79K3Hmjz.jpg,/foGkPxpw9h8zln81j63mix5B7m8.jpg,"Geralt of Rivia, a mutated monster-hunter for ...",https://www.youtube.com/watch?v=eb90gqGYP9c,tv,"Sci-Fi & Fantasy, Drama, Action & Adventure",66.0,24.0,8.09,2019,Netflix,/pbpMk2JmcoNnQwx5JGpXngfoWtp.jpg,United States of America,Sci-Fi & Fantasy,Drama
1,Two and a Half Men,2691,/xgfjxyV3g1S68opzuvG6G87muDp.jpg,/coHpOy9OJyFiHWeH36cSHMpLUHC.jpg,A hedonistic jingle writer's free-wheeling lif...,https://www.youtube.com/watch?v=h1YadOPGL-0,tv,Comedy,30.0,262.0,7.45,2003,Amazon Prime Video,/dQeAar5H991VYporEjUspolDarG.jpg,United States of America,Comedy,N/A
2,Dawn of the Planet of the Apes,119450,/kScdQEwS9jPEdnO23XjGAtaoRcT.jpg,/4TFCA5Lm5miseV3Wy02yxurEQL9.jpg,A group of scientists in San Francisco struggl...,https://www.youtube.com/watch?v=DpSaTrW4leg,movie,"Science Fiction, Action, Drama, Thriller",130.0,1.0,7.32,2014,Disney Plus,/97yvRBw1GzX7fXprcF80er19ot.jpg,United States of America,Science Fiction,Action
3,The Last Stop in Yuma County,1047020,/u6ae2MMh8PpSamO3cIttLM5B18v.jpg,/Y91KJmQ13jNEpOlNL9j6PPRpIO.jpg,While awaiting the next fuel truck at a middle...,https://www.youtube.com/watch?v=VfpnNciRQq8,movie,"Crime, Thriller",91.0,1.0,6.88,2024,N/A,N/A,United States of America,Crime,Thriller
4,Dark Matter,196322,/c6MRUtPk0nEPQ9FBD9RdRKt2rIm.jpg,/5fWxvjOUvtUoSmiMEpFl77V6KZV.jpg,Jason Dessen is abducted into an alternate ver...,https://www.youtube.com/watch?v=j6ucGt_Xp14,tv,"Drama, Mystery, Sci-Fi & Fantasy",50.0,9.0,7.9,2024,Apple TV Plus,/2E03IAZsX4ZaUqM7tXlctEPMGWS.jpg,United States of America,Drama,Mystery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4010,Locked Up,62455,/1kH9u5DkoDUuIlhFFWjz9VSlKhC.jpg,/iukGFUQQWUdgwTGGDLb1Pw3GoZP.jpg,"Set up to take the blame for corporate fraud, ...",https://www.youtube.com/watch?v=ceNKwOvSVm8,tv,"Drama, Crime",50.0,40.0,8.01,2015,N/A,N/A,Kingdom of Spain,Drama,Crime
4011,Titans,75450,/8e6QiSexmYKaiHGPvbhaFMmQEhc.jpg,/remQq0nfQoxY9fVglGGegHZjFxx.jpg,A team of young superheroes led by Nightwing (...,https://www.youtube.com/watch?v=9xIZoih_DaE,tv,"Action & Adventure, Sci-Fi & Fantasy, Drama",54.0,49.0,7.93,2018,Netflix,/pbpMk2JmcoNnQwx5JGpXngfoWtp.jpg,United States of America,Action & Adventure,Sci-Fi & Fantasy
4012,Ajin,65292,/fnHBwIKxwrFleSzvHzTqcAzDRM.jpg,/kOQkt6e18Fe5h9qnkd3PmffX7og.jpg,"17 years ago, immortals first appeared on the ...",https://www.youtube.com/watch?v=OO21Sq_CB0A,tv,"Animation, Action & Adventure, Mystery",24.0,26.0,8.01,2016,Netflix,/pbpMk2JmcoNnQwx5JGpXngfoWtp.jpg,Japan,Animation,Action & Adventure
4013,One Piece,37854,/cMD9Ygz11zjJzAovURpO75Qg7rT.jpg,/2rmK7mnchw9Xr3XdiTFSxTTLXqv.jpg,"Years ago, the fearsome Pirate King, Gol D. Ro...",https://www.youtube.com/watch?v=LrCmsOAgvxk,tv,"Action & Adventure, Comedy, Animation",24.0,1107.0,8.73,1999,Crunchyroll,/mXeC4TrcgdU6ltE9bCBCEORwSQR.jpg,Japan,Action & Adventure,Comedy


In [20]:
final_content.drop_duplicates(subset=['Content ID'],inplace=True)


final_content['Provider Name'] = final_content['Provider Name'].str.replace(r'\bAmazon Channel\b', '', regex=True)
final_content['Provider Name'] = final_content['Provider Name'].str.replace(r'\bApple TV Channel\b', '', regex=True)
final_content['Provider Name'] = final_content['Provider Name'].str.replace(r'\bNetflix basic with Ads\b', 'Netflix', regex=True)
final_content['Provider Name'] = final_content['Provider Name'].str.replace(r'\bParamount Plus\b', 'Paramount+', regex=True)
final_content['Provider Name'] = final_content['Provider Name'].str.replace(r'\STUDIOCANAL PRESENTS\b', 'Studiocanal Presents', regex=True)
final_content['Provider Name'] = df_content['Provider Name'].str.strip()

final_content['Number of Episodes'] = final_content['Number of Episodes'].replace('N/A', 0).astype(int)
final_content['Runtime (minutes)'] = final_content['Runtime (minutes)'].replace('N/A', 0).astype(int)
final_content['Total Runtime (minutes)'] = final_content['Number of Episodes'] * final_content['Runtime (minutes)']



In [21]:
final_content['Primary Genres'].unique()

array(['Sci-Fi & Fantasy', 'Comedy', 'Science Fiction', 'Crime', 'Drama',
       'Documentary', 'Animation', 'Thriller', 'Reality', 'Soap',
       'Mystery', 'Romance', 'Horror', 'Fantasy', 'Adventure', 'Action',
       'Music', 'Action & Adventure', 'War', 'Family', 'TV Movie', '',
       'Western', 'History', 'War & Politics', 'Kids', 'Talk'],
      dtype=object)

In [22]:
# The genres will be set into : Drama, Comedy, Action/Adventure, Science Fiction/Fantasy, Horror, Documentary/Other,Animation
def categorize_genre(genre):
    if 'Drama' in genre:
        return 'Drama'
    elif 'Animation' in genre:
        return 'Animation'
    elif 'Comedy' in genre:
        return 'Comedy'
    elif 'Action' in genre or 'Adventure' in genre:
        return 'Action/Adventure'
    elif 'Science Fiction' in genre or 'Fantasy' in genre:
        return 'Science Fiction/Fantasy'
    elif 'Horror' in genre:
        return 'Horror'
    else:
        return 'Documentary/Other'
       
final_content['Primary Genres'] = final_content['Primary Genres'].apply(categorize_genre)
final_content['Primary Genres'].unique()

array(['Science Fiction/Fantasy', 'Comedy', 'Documentary/Other', 'Drama',
       'Animation', 'Horror', 'Action/Adventure'], dtype=object)

In [23]:
# final_content['Origin Country'] = final_content['Origin Country'].str.replace(r'\[|\]', '', regex=True)
final_content['Origin Country'].unique()

def country_coordinates(country):
    geolocator = Nominatim(user_agent="geoapiExercises")
    try:
        location = geolocator.geocode(country)
        return location.latitude, location.longitude
    except:
        return None, None

In [24]:
final_content['Origin Country'].unique()

def country_coordinates(country):
    geolocator = Nominatim(user_agent="geoapiExercises")
    try:
        location = geolocator.geocode(country)
        return location.latitude, location.longitude
    except:
        return None, None
       
final_content['Latitude'], final_content['Longitude'] = zip(*final_content['Origin Country'].apply(country_coordinates))
final_content

,Title,Content ID,Poster,Backdrop,Overview,Trailer,Type,Genres,Runtime (minutes),Number of Episodes,Rating,Release Year,Provider Name,Provider Logo,Origin Country,Primary Genres,Secondary Genres,Total Runtime (minutes),Latitude,Longitude
0,The Witcher,71912,/cZ0d3rtvXPVvuiX22sP79K3Hmjz.jpg,/foGkPxpw9h8zln81j63mix5B7m8.jpg,"Geralt of Rivia, a mutated monster-hunter for ...",https://www.youtube.com/watch?v=eb90gqGYP9c,tv,"Sci-Fi & Fantasy, Drama, Action & Adventure",66,24,8.09,2019,Netflix,/pbpMk2JmcoNnQwx5JGpXngfoWtp.jpg,United States of America,Science Fiction/Fantasy,Drama,1584,39.783730,-100.445882
1,Two and a Half Men,2691,/xgfjxyV3g1S68opzuvG6G87muDp.jpg,/coHpOy9OJyFiHWeH36cSHMpLUHC.jpg,A hedonistic jingle writer's free-wheeling lif...,https://www.youtube.com/watch?v=h1YadOPGL-0,tv,Comedy,30,262,7.45,2003,Amazon Prime Video,/dQeAar5H991VYporEjUspolDarG.jpg,United States of America,Comedy,N/A,7860,39.783730,-100.445882
2,Dawn of the Planet of the Apes,119450,/kScdQEwS9jPEdnO23XjGAtaoRcT.jpg,/4TFCA5Lm5miseV3Wy02yxurEQL9.jpg,A group of scientists in San Francisco struggl...,https://www.youtube.com/watch?v=DpSaTrW4leg,movie,"Science Fiction, Action, Drama, Thriller",130,1,7.32,2014,Disney Plus,/97yvRBw1GzX7fXprcF80er19ot.jpg,United States of America,Science Fiction/Fantasy,Action,130,39.783730,-100.445882
3,The Last Stop in Yuma County,1047020,/u6ae2MMh8PpSamO3cIttLM5B18v.jpg,/Y91KJmQ13jNEpOlNL9j6PPRpIO.jpg,While awaiting the next fuel truck at a middle...,https://www.youtube.com/watch?v=VfpnNciRQq8,movie,"Crime, Thriller",91,1,6.88,2024,N/A,N/A,United States of America,Documentary/Other,Thriller,91,39.783730,-100.445882
4,Dark Matter,196322,/c6MRUtPk0nEPQ9FBD9RdRKt2rIm.jpg,/5fWxvjOUvtUoSmiMEpFl77V6KZV.jpg,Jason Dessen is abducted into an alternate ver...,https://www.youtube.com/watch?v=j6ucGt_Xp14,tv,"Drama, Mystery, Sci-Fi & Fantasy",50,9,7.9,2024,Apple TV Plus,/2E03IAZsX4ZaUqM7tXlctEPMGWS.jpg,United States of America,Drama,Mystery,450,39.783730,-100.445882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4001,Siren,71886,/k906XXqqFMT93v2WMkIOtUcEAlV.jpg,/uZNhVUocfi44FuyR3FmHOFLwio4.jpg,The coastal town of Bristol Cove is known for ...,N/A,tv,"Sci-Fi & Fantasy, Drama",42,36,7.73,2018,Disney Plus,/97yvRBw1GzX7fXprcF80er19ot.jpg,United States of America,Science Fiction/Fantasy,Drama,1512,39.783730,-100.445882
4003,FBI: Most Wanted,94372,/yZeV8gIPo27rL965Htb7LQJrFJU.jpg,/ukS3A6cAjzPI6LnojICqj5oG3EB.jpg,The Fugitive Task Force relentlessly tracks an...,https://www.youtube.com/watch?v=GjJSuHmeeyA,tv,"Crime, Drama",43,86,7.6,2020,Sky Go,/1UrT2H9x6DuQ9ytNhsSCUFtTUwS.jpg,United States of America,Documentary/Other,Drama,3698,39.783730,-100.445882
4008,Nine Perfect Strangers,88989,/oPcqsZIFHrv7HoGgFQnsGDiPulD.jpg,/lutscWuTri3WPa2ORXGObyzFFTV.jpg,At a boutique health-and-wellness resort that ...,https://www.youtube.com/watch?v=n-SExOBZ1uI,tv,"Drama, Mystery",55,11,7.08,2021,Amazon Prime Video,/dQeAar5H991VYporEjUspolDarG.jpg,United States of America,Drama,Mystery,605,39.783730,-100.445882
4012,Ajin,65292,/fnHBwIKxwrFleSzvHzTqcAzDRM.jpg,/kOQkt6e18Fe5h9qnkd3PmffX7og.jpg,"17 years ago, immortals first appeared on the ...",https://www.youtube.com/watch?v=OO21Sq_CB0A,tv,"Animation, Action & Adventure, Mystery",24,26,8.01,2016,NaN,/pbpMk2JmcoNnQwx5JGpXngfoWtp.jpg,Japan,Animation,Action & Adventure,624,36.574844,139.239418


# Data Retrieval from AniList

In [ ]:
import requests
import pandas as pd
import time

# Define the endpoint and query
endpoint = "https://graphql.anilist.co"
query = '''
query ($show: String) {
  Media (search: $show, type: ANIME) {
    title {
      romaji
      english
      native
    }
    genres
    startDate {
      year
    }
    format
    averageScore
  }
}
'''

# Initialize DataFrames
show_details_df = pd.DataFrame(columns=['Title', 'Rating', 'Year of Release', 'Type', 'Genres'])
skipped_shows_df = pd.DataFrame(columns=['Skipped Show'])

# Define constants for rate limiting
REQUESTS_PER_MINUTE = 90

# Process each title in the DataFrame
for index, title in enumerate(df3['Title']):
    variables = {"show": title}
    
    try:
        # Send the POST request
        response = requests.post(endpoint, json={"query": query, "variables": variables})
        response.raise_for_status()  # Raises stored HTTPError, if one occurred
        
        # Check rate limits
        remaining = int(response.headers.get('X-RateLimit-Remaining', 0))
        
        # Process response
        data = response.json()
        show = data["data"]["Media"]
        
        if show:
            show_details = {
                'Title': show["title"]['english'] or show["title"]['romaji'],
                'Rating': show['averageScore'] / 10 if show['averageScore'] else None,
                'Year of Release': show["startDate"]["year"],
                'Type': show["format"],
                'Genres': ', '.join(show["genres"])
            }
            show_details_df = pd.concat([show_details_df, pd.DataFrame([show_details])], ignore_index=True)
        else:
            skipped_shows_df = pd.concat([skipped_shows_df, pd.DataFrame([{'Skipped Show': title}])], ignore_index=True)
    
    except requests.exceptions.HTTPError as e:
        # Check if we hit the rate limit
        if response.status_code == 429:
            retry_after = int(response.headers.get('Retry-After', 60))
            print(f"Rate limit exceeded. Waiting for {retry_after} seconds.")
            time.sleep(retry_after)
        else:
            print(f"Request failed for {title}: {str(e)}")
            skipped_shows_df = pd.concat([skipped_shows_df, pd.DataFrame([{'Skipped Show': title}])], ignore_index=True)
    
    # Handle the pacing of requests based on remaining limit
    if remaining <= 1:
        print("Approaching rate limit. Pausing...")
        time.sleep(60)  # Sleep for 1 minute if the limit is reached
    else:
        time.sleep(1)  # Short sleep to spread out requests

# Output the results
print("Show Details:")
print(show_details_df)
print("\nSkipped Shows:")
print(skipped_shows_df)


In [25]:
final_content.to_csv('content_v4.csv')